In [1]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.config.architectures import BERTConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.architectures import BERT
from neural_blueprints.config.components.composite.projections import TabularProjectionConfig
from neural_blueprints.datasets import MaskedTabularDataset, TabularLabelDataset
from neural_blueprints.preprocess import TabularPreprocessor

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)

2026-01-22 16:25:17,340 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2026-01-22 16:25:17,340 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
2026-01-22 16:25:17,343 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'workclass' has 2799/5.73% NaN values; these will be encoded as 0.
2026-01-22 16:25:17,418 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'occupation' has 2809/5.75% NaN values; these will be encoded as 0.
2026-01-22 16:25:17,471 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'native-country' has 857/1.75% NaN values; these will be encoded as 0.


### Income Inference Accuracy

In [4]:
dataset = TabularLabelDataset(
    data=data,
    label_columns=['income'],              # Specify the label column for single-label classification
)

train_dataset, val_dataset = dataset.random_split([0.9, 0.1])

In [5]:
bert_config = BERTConfig(
    input_spec=(len(dataset.cardinalities), ),
    output_spec=(2, ),
    input_projection=TabularProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        hidden_dims=[32, 32],
        projection_dim=16,
        output_dim=[len(dataset.cardinalities), 16]
    ),
    output_projection=TabularProjectionConfig(
        input_dim=[len(dataset.cardinalities)*16],
        hidden_dims=[128, 64, 32, 16],
        output_cardinalities=[2],
    ),
    encoder_layers=2,
    dropout_p = 0.2,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = BERT(bert_config)
model.blueprint(batch_size=256, with_graph=False)

Layer (type:depth-idx)                                                      Output Shape              Param #
BERT                                                                        [256, 3]                  --
├─TabularInputProjection: 1-1                                               [256, 14, 16]             --
│    └─ModuleList: 2-1                                                      --                        --
│    │    └─NumericalProjection: 3-1                                        [256, 16]                 1,776
│    │    └─DiscreteProjection: 3-2                                         [256, 16]                 3,152
│    │    └─NumericalProjection: 3-3                                        [256, 16]                 1,776
│    │    └─DiscreteProjection: 3-4                                         [256, 16]                 3,376
│    │    └─DiscreteProjection: 3-5                                         [256, 16]                 3,376
│    │    └─DiscreteProjection: 3-6

In [6]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='cross_entropy',
        optimizer='adam',
        early_stopping_patience=5,
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=256
    )
)
trainer.train(train_dataset, val_dataset, epochs=5, visualize=True)
trainer.predict(val_dataset)

2026-01-22 16:25:17,659 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Training Epochs:  20%|██        | 1/5 [00:03<00:15,  3.93s/epoch]

Epoch 1/5, Training Loss: 0.9286, Validation Loss: 0.8029


Training Epochs:  40%|████      | 2/5 [00:07<00:11,  3.88s/epoch]

Epoch 2/5, Training Loss: 0.7592, Validation Loss: 0.7080


...

Training Epochs: 100%|██████████| 5/5 [00:19<00:00,  3.87s/epoch]
2026-01-22 16:25:36,995 - neural_blueprints.utils.trainer - INFO - Training completed in 19.34 seconds.
2026-01-22 16:25:36,995 - neural_blueprints.utils.trainer - INFO - Best validation loss: 6.8129e-01


Epoch 5/5, Training Loss: 0.6867, Validation Loss: 0.6813


2026-01-22 16:25:37,227 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.13 seconds.


Classification Accuracy: 0.8514


0.851381780962129

### Masked Dataset Inference Accuracy

In [7]:
# Create dataset
dataset = MaskedTabularDataset(
    data, 
    mask_prob=0.35
)

train_dataset, val_dataset = dataset.random_split([0.9, 0.1])

In [8]:
bert_config = BERTConfig(
    input_spec=(len(dataset.cardinalities), ),
    output_spec=(len(dataset.cardinalities), ),
    input_projection=TabularProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        hidden_dims=[32, 32],
        projection_dim=16,
        output_dim=[len(dataset.cardinalities), 16]
    ),
    output_projection=TabularProjectionConfig(
        input_dim=[len(dataset.cardinalities), 16],
        hidden_dims=[16, 16],
        output_cardinalities=dataset.cardinalities,
    ),
    encoder_layers=12,
    dropout_p = 0.1,
    normalization = "layernorm",
    activation = "gelu",
    final_activation = None
)

model = BERT(bert_config)
model.blueprint(batch_size=256, with_graph=False)

Layer (type:depth-idx)                                                      Output Shape              Param #
BERT                                                                        [256, 1]                  --
├─TabularInputProjection: 1-1                                               [256, 15, 16]             --
│    └─ModuleList: 2-1                                                      --                        --
│    │    └─NumericalProjection: 3-1                                        [256, 16]                 1,776
│    │    └─DiscreteProjection: 3-2                                         [256, 16]                 3,152
│    │    └─NumericalProjection: 3-3                                        [256, 16]                 1,776
│    │    └─DiscreteProjection: 3-4                                         [256, 16]                 3,376
│    │    └─DiscreteProjection: 3-5                                         [256, 16]                 3,376
│    │    └─DiscreteProjection: 3-6

In [9]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='masked_reconstruction',
        optimizer='adam',
        early_stopping_patience=5,
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=256
    )
)
trainer.train(train_dataset, val_dataset, epochs=1, visualize=True)
trainer.predict(val_dataset)

2026-01-22 16:25:37,423 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu
Training Epochs: 100%|██████████| 1/1 [00:35<00:00, 35.48s/epoch]
2026-01-22 16:26:12,903 - neural_blueprints.utils.trainer - INFO - Training completed in 35.48 seconds.
2026-01-22 16:26:12,903 - neural_blueprints.utils.trainer - INFO - Best validation loss: 6.6283e+00


Epoch 1/1, Training Loss: 7.0971, Validation Loss: 6.6283


2026-01-22 16:26:14,045 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 1.
2026-01-22 16:26:14,046 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 3.
2026-01-22 16:26:14,046 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 4.
2026-01-22 16:26:14,048 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 7.
2026-01-22 16:26:14,048 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 8.
2026-01-22 16:26:14,049 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 9.
2026-01-22 16:26:14,050 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 13.
2026-01-22 16:26:14,050 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature c

Feature Column 0:
Predicted attribute values: [0.25452316 0.31773153 0.25921792 0.37159148 0.24145827]
True attribute values: [0.09589041 0.2739726  0.09589041 0.53424656 0.26027396]
Accuracy: 0.1878

Feature Column 1:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [2. 4. 4. 6. 0.]
Accuracy: 0.7074

Feature Column 2:
Predicted attribute values: [0.11696593 0.12102088 0.12200111 0.1207561  0.11615397]
True attribute values: [0.14830105 0.17925736 0.07473302 0.06044252 0.12447002]
Accuracy: 0.5833

Feature Column 3:
Predicted attribute values: [16 16 16 16 16]
True attribute values: [12. 16. 16. 16. 13.]
Accuracy: 0.2268

Feature Column 4:
Predicted attribute values: [5 5 5 5 5]
True attribute values: [ 2. 16.  2.  5. 16.]
Accuracy: 0.1714

Feature Column 5:
Predicted attribute values: [5 5 3 5 5]
True attribute values: [5. 5. 3. 6. 1.]
Accuracy: 0.5124

Feature Column 6:
Predicted attribute values: [4 3 4 4 4]
True attribute values: [3. 3. 0. 4. 3.]
Accuracy: 0.1021

Fea

{'avg_discrete_accuracy': np.float64(0.5301402334269433),
 'avg_continuous_accuracy': np.float64(0.2725969032296415),
 'overall_avg_accuracy': np.float64(0.44429245669450934)}